In [25]:
from tagmatch.vec_db import Embedder
import numpy as np
from tagmatch.vec_db import VecDB
import itertools
import pickle
from create_dummy_tags import tags
import nanopq
from pydantic_settings import BaseSettings
class Settings(BaseSettings):
    model_name: str
    cache_dir: str
    qdrant_host: str
    qdrant_port: int
    qdrant_collection: str
    reduced_embed_dim: int

    class Config:
        env_file = ".env"
settings=Settings()

/opt/homebrew/Caskroom/miniconda/base/envs/tagsearch/lib/python3.11/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ('settings_',)`.
  warnings.warn(


In [26]:
with open("additional_tags.txt") as f:
    additional_tags = f.readlines()
    additional_tags = [t.split(",") for t in additional_tags]
    additional_tags = list(itertools.chain.from_iterable(additional_tags))
    additional_tags = [t.lstrip().replace(r'\n','') for t in additional_tags]

In [31]:
extended_tags = tags + additional_tags

In [32]:
embedder = Embedder(model_name=settings.model_name,cache_dir=settings.cache_dir)
tag_embeddings = [embedder.embed(tag) for tag in extended_tags]
embed_vecs=np.asarray(tag_embeddings)

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 55043.36it/s]


In [33]:
pq = nanopq.PQ(M=64)
pq.fit(embed_vecs)

M: 64, Ks: 256, metric : <class 'numpy.uint8'>, code_dtype: l2
iter: 20, seed: 123
Training the subspace: 0 / 64
Training the subspace: 1 / 64
Training the subspace: 2 / 64
Training the subspace: 3 / 64
Training the subspace: 4 / 64
Training the subspace: 5 / 64
Training the subspace: 6 / 64
Training the subspace: 7 / 64
Training the subspace: 8 / 64
Training the subspace: 9 / 64
Training the subspace: 10 / 64
Training the subspace: 11 / 64
Training the subspace: 12 / 64
Training the subspace: 13 / 64
Training the subspace: 14 / 64
Training the subspace: 15 / 64
Training the subspace: 16 / 64
Training the subspace: 17 / 64
Training the subspace: 18 / 64
Training the subspace: 19 / 64
Training the subspace: 20 / 64


/opt/homebrew/Caskroom/miniconda/base/envs/tagsearch/lib/python3.11/site-packages/nanopq/pq.py:123: UserWarning: One of the clusters is empty. Re-run kmeans with a different initialization.
  self.codewords[m], _ = kmeans2(vecs_sub, self.Ks, iter=iter, minit=minit)


Training the subspace: 21 / 64
Training the subspace: 22 / 64
Training the subspace: 23 / 64
Training the subspace: 24 / 64
Training the subspace: 25 / 64
Training the subspace: 26 / 64
Training the subspace: 27 / 64
Training the subspace: 28 / 64
Training the subspace: 29 / 64
Training the subspace: 30 / 64
Training the subspace: 31 / 64
Training the subspace: 32 / 64
Training the subspace: 33 / 64
Training the subspace: 34 / 64
Training the subspace: 35 / 64
Training the subspace: 36 / 64
Training the subspace: 37 / 64
Training the subspace: 38 / 64
Training the subspace: 39 / 64
Training the subspace: 40 / 64
Training the subspace: 41 / 64
Training the subspace: 42 / 64
Training the subspace: 43 / 64
Training the subspace: 44 / 64
Training the subspace: 45 / 64
Training the subspace: 46 / 64
Training the subspace: 47 / 64
Training the subspace: 48 / 64
Training the subspace: 49 / 64
Training the subspace: 50 / 64
Training the subspace: 51 / 64
Training the subspace: 52 / 64
Training

In [53]:
quantized_embeddings = pq.encode(embed_vecs[:len(tags)])

Encoding the subspace: 0 / 64
Encoding the subspace: 1 / 64
Encoding the subspace: 2 / 64
Encoding the subspace: 3 / 64
Encoding the subspace: 4 / 64
Encoding the subspace: 5 / 64
Encoding the subspace: 6 / 64
Encoding the subspace: 7 / 64
Encoding the subspace: 8 / 64
Encoding the subspace: 9 / 64
Encoding the subspace: 10 / 64
Encoding the subspace: 11 / 64
Encoding the subspace: 12 / 64
Encoding the subspace: 13 / 64
Encoding the subspace: 14 / 64
Encoding the subspace: 15 / 64
Encoding the subspace: 16 / 64
Encoding the subspace: 17 / 64
Encoding the subspace: 18 / 64
Encoding the subspace: 19 / 64
Encoding the subspace: 20 / 64
Encoding the subspace: 21 / 64
Encoding the subspace: 22 / 64
Encoding the subspace: 23 / 64
Encoding the subspace: 24 / 64
Encoding the subspace: 25 / 64
Encoding the subspace: 26 / 64
Encoding the subspace: 27 / 64
Encoding the subspace: 28 / 64
Encoding the subspace: 29 / 64
Encoding the subspace: 30 / 64
Encoding the subspace: 31 / 64
Encoding the subsp

In [52]:
vec_db = VecDB(
    host="http://localhost",
    port=6333,
    collection="pca_train_test_reduced",
    vector_size=64,
)

In [54]:
for name,vec in zip(tags,quantized_embeddings):
    vec_db.store(vec,{"name":name})

In [55]:
reduced_vector_matches = [vec_db.find_closest(query_vector, 5) for query_vector in quantized_embeddings]

In [56]:
reduced_vector_matches[0]

[ScoredPoint(id=12577971619524015535, version=0, score=1.0000001, payload={'name': 'Apollo 11'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13597808653146884023, version=168, score=0.86957663, payload={'name': 'White Dwarf'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=10869594146043148504, version=126, score=0.86057645, payload={'name': 'Space Policy'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13339345376839178795, version=72, score=0.83821887, payload={'name': 'Artificial Gravity'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12404555467008947710, version=165, score=0.8297652, payload={'name': 'Universe'}, vector=None, shard_key=None, order_value=None)]

In [58]:
vec_db_full = VecDB(
    host="http://localhost",
    port=6333,
    collection="train_test",
    vector_size=embedder.embedding_dim,
)

In [59]:
for name,vec in zip(tags,embed_vecs):
    vec_db_full.store(vec,{"name":name})

In [60]:

vector_matches = [vec_db_full.find_closest(query_vector, 5) for query_vector in embed_vecs]


In [61]:
vector_matches[0]

[ScoredPoint(id=10590435730962342384, version=0, score=0.9999999, payload={'name': 'Apollo 11'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=10938686976877578232, version=46, score=0.7471009, payload={'name': 'Moon Mission'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=9329965740516622676, version=1, score=0.72500336, payload={'name': 'Moon'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=12340914482847751841, version=80, score=0.7157799, payload={'name': 'Lunar Base'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=13599961305190117704, version=14, score=0.6981933, payload={'name': 'NASA'}, vector=None, shard_key=None, order_value=None)]

In [61]:
pickle.dump(pca,open("pca.pkl","wb"))